In [1]:
import pulp
import pandas as pd
import numpy as np

def optimize_containers(df, wt, vt):
    # 获取唯一的目的地
    destinations = df['d'].unique()

    # 创建问题
    prob = pulp.LpProblem("Container_Optimization", pulp.LpMinimize)

    # 计算需要的最小集装箱数（向上取整）
    min_containers_volume = int(np.ceil(sum(df['v'] * df['n']) / vt))
    min_containers_weight = int(np.ceil(sum(df['w'] * df['n']) / wt))
    min_containers = max(min_containers_volume, min_containers_weight)
    max_containers = min_containers * 2

    print(f"按体积计算的最小集装箱数: {min_containers_volume}")
    print(f"按重量计算的最小集装箱数: {min_containers_weight}")
    print(f"理论最小集装箱数: {min_containers}")

    # 决策变量
    # y[k]: 是否使用第k个集装箱
    y = pulp.LpVariable.dicts("container",
                             range(max_containers),
                             cat='Binary')

    # x[i,k]: 货物i分配到集装箱k的数量
    x = pulp.LpVariable.dicts("allocation",
                             ((i, k) for i in range(len(df))
                              for k in range(max_containers)),
                             lowBound=0,
                             cat='Integer')

    # d[k,dest]: 集装箱k是否用于目的地dest
    d = pulp.LpVariable.dicts("destination",
                             ((k, dest) for k in range(max_containers)
                              for dest in destinations),
                             cat='Binary')

    # 目标函数：最小化使用的集装箱数量
    prob += pulp.lpSum(y[k] for k in range(max_containers))

    # 约束条件
    # 1. 每种货物必须被完全分配
    for i in range(len(df)):
        prob += pulp.lpSum(x[i,k] for k in range(max_containers)) == df.iloc[i]['n']

    # 2. 每个集装箱的重量和体积限制
    for k in range(max_containers):
        # 重量约束
        prob += pulp.lpSum(x[i,k] * df.iloc[i]['w'] for i in range(len(df))) <= wt * y[k]
        # 体积约束
        prob += pulp.lpSum(x[i,k] * df.iloc[i]['v'] for i in range(len(df))) <= vt * y[k]

    # 3. 目的地约束（新版本）
    M = sum(df['n'])  # 一个足够大的数
    for k in range(max_containers):
        # 每个集装箱只能用于一个目的地
        prob += pulp.lpSum(d[k,dest] for dest in destinations) <= 1

        # 如果集装箱被使用，必须指定一个目的地
        prob += pulp.lpSum(d[k,dest] for dest in destinations) >= y[k]

        # 如果装了某个目的地的货物，对应的d[k,dest]必须为1
        for dest in destinations:
            dest_items = df[df['d'] == dest].index
            prob += pulp.lpSum(x[i,k] for i in dest_items) <= M * d[k,dest]

        # 如果d[k,dest]=1，只能装该目的地的货物
        for dest in destinations:
            other_dest_items = df[df['d'] != dest].index
            prob += pulp.lpSum(x[i,k] for i in other_dest_items) <= M * (1 - d[k,dest])

    # 求解
    print("开始求解...")
    solver = pulp.PULP_CBC_CMD(msg=True, timeLimit=120)
    status = prob.solve(solver)
    print(f"求解状态: {pulp.LpStatus[status]}")

    if status != pulp.LpStatusOptimal:
        print("未找到最优解！")
        return None

    # 获取结果
    results = []
    for k in range(max_containers):
        if pulp.value(y[k]) > 0.5:  # 使用了这个集装箱
            container_items = []
            container_weight = 0
            container_volume = 0
            container_dest = None

            for i in range(len(df)):
                qty = int(pulp.value(x[i,k]))
                if qty > 0:
                    item_weight = df.iloc[i]['w'] * qty
                    item_volume = df.iloc[i]['v'] * qty
                    container_items.append({
                        'item_index': i,
                        'destination': df.iloc[i]['d'],
                        'quantity': qty,
                        'weight': item_weight,
                        'volume': item_volume
                    })
                    container_weight += item_weight
                    container_volume += item_volume
                    container_dest = df.iloc[i]['d']

            if container_items:  # 只添加非空的集装箱
                results.append({
                    'container_id': k,
                    'destination': container_dest,
                    'total_weight': container_weight,
                    'total_volume': container_volume,
                    'items': container_items
                })

    return results

# 测试数据
data = {
    'v': [10, 8, 15, 20, 5, 12],
    'w': [2, 3, 4, 5, 1, 6],
    'd': ['PortX', 'PortY', 'PortX', 'PortY', 'PortX', 'PortY'],
    'n': [50, 70, 40, 60, 80, 90],
}
df = pd.DataFrame(data)

# 计算每个目的地的总量
print("\n目的地货物统计：")
for dest in df['d'].unique():
    dest_df = df[df['d'] == dest]
    total_volume = sum(dest_df['v'] * dest_df['n'])
    total_weight = sum(dest_df['w'] * dest_df['n'])
    print(f"{dest}: 总体积={total_volume}, 总重量={total_weight}")

# 运行优化
print("\n开始优化...")
results = optimize_containers(df, wt=22, vt=71)

if results:
    print(f"\n总共需要 {len(results)} 个集装箱")
    for container in results:
        print(f"\n集装箱 {container['container_id']} (目的地: {container['destination']}):")
        print(f"总重量: {container['total_weight']:.2f}/{wt}")
        print(f"总体积: {container['total_volume']:.2f}/{vt}")
        print("装载的货物:")
        for item in container['items']:
            print(f"- 货物 {item['item_index']}: "
                  f"{item['quantity']}件, "
                  f"重量={item['weight']:.2f}, "
                  f"体积={item['volume']:.2f}")

ModuleNotFoundError: No module named 'pulp'